## Import libraries and Load data

In [32]:
import pandas as pd
import numpy as np

## Load Datasets

In [33]:
players = pd.read_csv("../data/male_players.csv", low_memory=False)
teams = pd.read_csv("../data/male_teams.csv", )

In [34]:
players["club_team_id"].head()

0        73.0
1        10.0
2        10.0
3    112893.0
4       607.0
Name: club_team_id, dtype: float64

## Merge Datasets

In [35]:
teams = teams.rename(columns={"team_id": "club_team_id"})

In [36]:
df = pd.merge(players, teams, on="club_team_id", how="left", suffixes=("_player", "_team"))

In [37]:
print(f"Total row: {df.shape[0]}")
print(f"Total column: {df.shape[1]}")

Total row: 1417641
Total column: 162


## Filter Latest Version and Remove Duplicates

In [38]:
print(df["fifa_version_player"].unique())

[24. 23. 22. 21. 20. 19. 18. 17. 16. 15.]


In [39]:
df_latest = df[df["fifa_version_player"] == df["fifa_version_player"].max()].copy()

In [40]:
df_final = df_latest.sort_values("overall_player", ascending=False).drop_duplicates(subset=["short_name", "age"])

In [41]:
df_final[df_final["short_name"] == "K. Mbappé"]

,player_id,player_url,fifa_version_player,fifa_update_player,update_as_of_player,short_name,long_name,player_positions,overall_player,potential,...,off_corners,off_free_kicks,build_up_play_speed,build_up_play_dribbling,build_up_play_passing,build_up_play_positioning,chance_creation_passing,chance_creation_crossing,chance_creation_shooting,chance_creation_positioning
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Select Relevant Features and Handle Missing Values

In [42]:
selected_cols = [
    'player_id', 'short_name', 'player_positions', 'overall_player', 'potential',
    'age', 'value_eur', 'wage_eur', 'league_id_player', 'league_name_player', 'league_level_player',
    'club_name', 'player_url', 'team_url',
    'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic'
]

In [44]:
df_scout = df_final[selected_cols].copy()

In [45]:
df_scout = df_scout.rename(columns={
    'overall_player': 'overall',
    'league_name_player': 'league_name',
    'league_id_player': 'league_id',
    'league_level_player': 'league_level'
})

In [46]:
df_scout = df_scout.dropna(subset=['value_eur', 'player_url', 'club_name'])

## Feature Engineering

In [47]:
df_scout['unique_league'] = df_scout['league_name'] + "_" + df_scout['league_id'].astype(str).str.replace('.0', '', regex=False)

league_quality = df_scout.groupby('unique_league')['overall'].mean().sort_values(ascending=False)

top_league_avg = league_quality.max()

league_coefs = league_quality / top_league_avg

df_scout["league_coef"] = df_scout["unique_league"].map(league_coefs)

print(league_coefs.head(10))

unique_league
Premier League_13    1.000000
La Liga_53           0.995580
Serie A_31           0.990333
Serie A_7            0.984111
Bundesliga_19        0.980666
Super League_63      0.977394
Fortuna Liga_319     0.973007
Ligue 1_16           0.970450
NB I._64             0.950622
1. HNL_317           0.936867
Name: overall, dtype: float64


In [48]:
df_scout["league_coef"] = df_scout["unique_league"].map(league_coefs)

In [49]:
df_scout["unique_league"].head(10)

0                 Ligue 1_16
10         Premier League_13
20         Premier League_13
30    Major League Soccer_39
34            Pro League_350
44                La Liga_53
54                La Liga_53
64             Bundesliga_19
74                La Liga_53
84         Premier League_13
Name: unique_league, dtype: object

In [50]:
df_scout[["league_name", "league_id", 'unique_league']].head(10)

,league_name,league_id,unique_league
0,Ligue 1,16.0,Ligue 1_16
10,Premier League,13.0,Premier League_13
20,Premier League,13.0,Premier League_13
30,Major League Soccer,39.0,Major League Soccer_39
34,Pro League,350.0,Pro League_350
44,La Liga,53.0,La Liga_53
54,La Liga,53.0,La Liga_53
64,Bundesliga,19.0,Bundesliga_19
74,La Liga,53.0,La Liga_53
84,Premier League,13.0,Premier League_13


## Normalize All Technical Attributes

In [51]:
technical_features = ['overall', 'potential', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']

for feature in technical_features:
    df_scout[f'norm_{feature}'] = df_scout[feature] * df_scout['league_coef']

In [52]:
#Drop goalkeepers and players missing pace stats
df_scout = df_scout.dropna(subset=['norm_pace'])

In [53]:
df_scout.to_csv("../data/cleaned_players.csv", index=False)

In [58]:
df_scout.columns

Index(['player_id', 'short_name', 'player_positions', 'overall', 'potential',
       'age', 'value_eur', 'wage_eur', 'league_id', 'league_name',
       'league_level', 'club_name', 'player_url', 'team_url', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'unique_league', 'league_coef', 'norm_overall', 'norm_potential',
       'norm_pace', 'norm_shooting', 'norm_passing', 'norm_dribbling',
       'norm_defending', 'norm_physic'],
      dtype='object')